# Train the Fully Convolutional Network (FCN) baseline
Train an FCN-32s network as defined by [Long (2014)](#References) using limited hyper parameter tuning. Its performance serves as a baseline DECiSION and VOLVuLuS are compared against.

# Set seeds and import packages
Setting the seeds first is meant to achieve reproducibility, though this goal is not entirely achieved.

In [1]:
# Set Python, NumPy and TensorFlow seeds
RANDOM_STATE = 42
from numpy.random import seed
seed(RANDOM_STATE)

from tensorflow import set_random_seed
set_random_seed(RANDOM_STATE)

import random
random.seed = RANDOM_STATE

# Model and training settings
import FCN_baseline_settings as settings

# Toolkit imports
from dltoolkit.utils.generic import model_architecture_to_file, model_summary_to_file, list_images
from dltoolkit.nn.segment import FCN32_NN# , FCN32_VGG16_NN
from dltoolkit.utils.visual import plot_training_history, plot_roc_curve,\
    plot_precision_recall_curve, print_confusion_matrix, print_classification_report
from dltoolkit.iomisc import HDF5Generator_Segment

from thesis_common import convert_img_to_pred, convert_pred_to_img, create_hdf5_db,\
    show_image, read_images, read_groundtruths, print_training_info
from thesis_metric_loss import dice_coef_threshold, weighted_pixelwise_crossentropy_loss, evaluate_model

# Keras imports
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, TensorBoard,\
    ReduceLROnPlateau
from keras.optimizers import Adam

# scikit-learn imports
from sklearn.model_selection import train_test_split

# Other imports
import numpy as np
import os, cv2, time, progressbar
import matplotlib.pyplot as plt
%matplotlib inline

/Users/geoff/anaconda3/envs/ML3-DL-OPENCV/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Change how TensorFlow allocates GPU memory
Setting `gpu_options.allow_growth` to `True` means TensorFlow will allocate GPU memory as needed rather than using all available memory from the start. This enables monitoring of actual memory usage and determining how close the notebook gets to running out of memory. This has no effect on non-GPU machines.

In [2]:
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
 
# Only allow a percentage of the GPU memory to be allocated
# config.gpu_options.per_process_gpu_memory_fraction = 0.5
 
# Create a session with the above options specified
k.tensorflow_backend.set_session(tf.Session(config=config))

# Determine training settings
The variables below determine how the model will be trained:

- `USE_GENERATORS`: when set to `True` the training process will use data generators, which is typically what will be required given the size of the models being trained and the size of the images. Set to `False` to (try to) load all training data into memory.
- `USE_VALIDATION_SET`: set to `True` to use a validation set during training, which will be the case most of the time. set to `False` to not use a validation set, e.g. during pipeline development/validation.
- `IS_DEVELOPMENT`: set to `True` to re-create the HDF5 files every time training is executed, `False` otherwise.

**Note** if `TRN_TRAIN_VAL_SPLIT` is set to 0, a validation set will *not* be created, even if `USE_VALIDATION_SET` is set to `True`. In fact, `USE_VALIDATION_SET` will be set to `False`.

In [3]:
USE_GENERATORS = True
USE_VALIDATION_SET = True
IS_DEVELOPMENT = True

if not USE_VALIDATION_SET:
    # If no validation set is to be used override the split value
    settings.TRN_TRAIN_VAL_SPLIT = 0.0

# Convert individual JPGs to HDF5 data sets
This function converts individual JPG files to HDF5 data sets. It creates up to four HDF5 files depending on whether a portion of the files is set aside as a validation set using the `settings.TRN_TRAIN_VAL_SPLIT` parameter.

This conversion only has to be performed once rather than prior to every training run. However, it takes very little time to run. **Note**: if any changes are made to image pre-processing applied to MRI images and/or their ground truths, the conversion MUST be executed prior to the next training run. Failing to do this means training will be performed on existing HDF5 data sets that do not include the revised pre-processing.

The folder stucture is shown below. MRI images (i.e. the individial JPG files, one for each *slice*) are located in the `images` folder. Each patient has a separate subfolder. Ground truths are located in the `groundtruths` folder, again each patient has a separate subfolder. The association between MRI image and ground truth is based on the name of the patient's subfolder **and** the alphabetically sorted file name. An MRI image/ground truth pair constitutes one training sample. The resulting HDF5 data sets are stored in the `images` and `ground truths` subfolders. The training data sets are called `train_images.h5` and `train_groundtruths.h5`. The validation sets, if they were created, are called `val_images.h5` and `val_groundtruths.h5`. 

```
---training
|     |--- images
|     |      |---patient1
|     |      |      |--- image1
|     |      |      |--- image2
|     |      |      ...
|     |      |      |--- imageM
|     |      |---patient2
|     |      ...
|     |      |---patientN
|     |--- groundtruths
|     |      |---patient1
|     |      |      |--- groundtruth1
|     |      |      |--- groundtruth2
|     |      |      ...
|     |      |      |--- groundtruthM
|     |      |---patient2
|     |      ...
|     |      |---patientN

```

In [4]:
def perform_hdf5_conversion(settings):
    """Convert the individual .jpg files (images and ground truths) to four .hdf5 files:
    - Two with the training set (train_images.h5 and train_groundtruths.h5)
    - Two with the validation set (val_images.h5 and val_groundtruths.h5)
    
    If TRN_TRAIN_VAL_SPLIT is set to zero NO validation data sets are generated.
    
    Returns the path to the HDF5 files and the number of patient folders processed.
    """
    # Prepare the path to the training images and ground truths
    img_path = os.path.join(settings.TRAINING_PATH, settings.FLDR_IMAGES)
    msk_path = os.path.join(settings.TRAINING_PATH, settings.FLDR_GROUND_TRUTH)

    # Create a list of paths to the individual patient folders
    patient_fld_imgs = sorted([os.path.join(img_path, e.name)
                               for e in os.scandir(img_path) if e.is_dir()])
    patient_fld_masks = sorted([os.path.join(msk_path, e.name)
                                for e in os.scandir(msk_path) if e.is_dir()])

    # Obtain a list of paths to the training images and ground truths for each patient
    img_list = []
    msk_list = []
    for patient_ix, (p_fld_imgs, p_fld_masks) in enumerate(zip(patient_fld_imgs,
                                                               patient_fld_masks)):
        img_list.extend(sorted(list(list_images(basePath=p_fld_imgs,
                                                validExts=settings.IMG_EXTENSION)))
                        [settings.SLICE_START:settings.SLICE_END])
        msk_list.extend(sorted(list(list_images(basePath=p_fld_masks,
                                                validExts=settings.IMG_EXTENSION)))
                        [settings.SLICE_START:settings.SLICE_END])

    assert(len(img_list) == len(msk_list))

    # Split the training set into a training and validation set. Consecutive slices no longer
    # belong to the same patient
    train_img, val_img, train_msk, val_msk = train_test_split(img_list, msk_list,
                                                              test_size=settings.TRN_TRAIN_VAL_SPLIT,
                                                              random_state=settings.RANDOM_STATE,
                                                              shuffle=True)
    
    # Create the HDF5 data sets
    output_paths = []

    # Training images
    output_paths.append(create_hdf5_db(train_img, "train", img_path,
                                       (settings.IMG_HEIGHT,
                                        settings.IMG_WIDTH,
                                        settings.IMG_CHANNELS),
                                       key=settings.HDF5_KEY,
                                       ext=settings.HDF5_EXT,
                                       settings=settings))

    # Training ground truths
    output_paths.append(create_hdf5_db(train_msk, "train", msk_path,
                                       (settings.IMG_HEIGHT,
                                        settings.IMG_WIDTH,
                                        settings.IMG_CHANNELS),
                                       key=settings.HDF5_KEY,
                                       ext=settings.HDF5_EXT,
                                       settings=settings,
                                       is_mask=True))

    # Validation images
    output_paths.append(create_hdf5_db(val_img, "val", img_path,
                                       (settings.IMG_HEIGHT,
                                        settings.IMG_WIDTH,
                                        settings.IMG_CHANNELS),
                                       key=settings.HDF5_KEY,
                                       ext=settings.HDF5_EXT,
                                       settings=settings))

    # Validation ground truths
    output_paths.append(create_hdf5_db(val_msk, "val", msk_path,
                                       (settings.IMG_HEIGHT,
                                        settings.IMG_WIDTH,
                                        settings.IMG_CHANNELS),
                                       key=settings.HDF5_KEY,
                                       ext=settings.HDF5_EXT,
                                       settings=settings,
                                       is_mask=True))

    return output_paths, len(patient_fld_imgs)

In [5]:
if IS_DEVELOPMENT:
    print("\n--- Converted images to HDF5, full paths:\n")
    hdf5_paths, num_patients = perform_hdf5_conversion(settings)
    
    if settings.TRN_TRAIN_VAL_SPLIT == 0:
        print("\nValidation set NOT created.")
        USE_VALIDATION_SET = False
    else:
        print("\nCreated a {:.2f} split.".format(settings.TRN_TRAIN_VAL_SPLIT))
else:
    # Use existing HDF5 data sets
    hdf5_paths = ["../data/MSC8002/training/train_images.h5",
                  "../data/MSC8002/training/train_groundtruths.h5",
                  "../data/MSC8002/training/val_images.h5",
                  "../data/MSC8002/training/val_groundtruths.h5"]
    num_patients = None

Creating HDF5 database  17% |######                             | ETA:  0:00:00


--- Converted images to HDF5, full paths:

../data/MSC8002/training/train_images.h5


Creating HDF5 database 100% |###################################| Time: 0:00:00
Creating HDF5 database 100% |###################################| Time: 0:00:00
Creating HDF5 database N/A% |                                  | ETA:  --:--:--

../data/MSC8002/training/train_groundtruths.h5
../data/MSC8002/training/val_images.h5


Creating HDF5 database 100% |###################################| Time: 0:00:00
Creating HDF5 database 100% |###################################| Time: 0:00:00


../data/MSC8002/training/val_groundtruths.h5

Created a 0.20 split.


# Set the class distribution
Assigning a higher weight to the positive class (i.e. blood vessels) means the model will pay "more attention" to that class. This is useful in the current class imbalance scenario because the number of background (i.e. non-blood vessel) pixels far exceed the number of blood vessel pixels. Without setting a different class weight for the blood vessel class the model would simply assign the background class to all pixels to achieve a low loss.

In [6]:
class_weights = [settings.CLASS_WEIGHT_BACKGROUND, settings.CLASS_WEIGHT_BLOODVESSEL]
print("Class distribution: {}".format(class_weights))

Class distribution: [1.0, 10.0]


# Create the FCN model
Instantiate the FCN-32s model. **Warning**: Changing the model and/or its parameters will change the name of the file the trained model will be saved to. Make sure to update the `FCN_baseline_test_ipynb` notebook accordingly to ensure it uses the correct saved model. **Note**: contrary to the paper, the model does not load weights pre-trained on, for example, ImageNet.

In [7]:
fcn = FCN32_NN(img_height=settings.IMG_HEIGHT,
# fcn = FCN32_VGG16_NN(img_height=settings.IMG_HEIGHT,
               img_width=settings.IMG_WIDTH,
               img_channels=settings.IMG_CHANNELS,
               num_classes=settings.NUM_CLASSES,
               dropout_rate=settings.TRN_DROPOUT_RATE)

# Crop 16 pixels on all sides to get 256x256 images
model = fcn.build_model(crop=16, use_bn=True)
# model = fcn.build_model(crop=16)

# Create paths
This cell just creates a few paths used later to save training output (e.g. the model architecture, training results and so on).

In [8]:
prefix = "FCN_baseline_" + unet.title + "_BS" + "{:03}".format(settings.TRN_BATCH_SIZE)

model_path = os.path.join(settings.MODEL_PATH, prefix + ".model")
summ_path = os.path.join(settings.OUTPUT_PATH, prefix + "_model_summary.txt")
csv_path = os.path.join(settings.OUTPUT_PATH, prefix + "_training.csv")

# Save/print model architecture information
Save the model's architecture to a file, print it in the cell below and save a diagram to disk.

In [ ]:
model.summary()
model_summary_to_file(model, summ_path)
model_architecture_to_file(unet.model, os.path.join(settings.OUTPUT_PATH, prefix))

# Compile the model
Set the loss function, optimiser and metric and compile the model. 

In [ ]:
# Set the optimiser, loss function and metrics
opt = Adam(amsgrad=True)
metrics = [dice_coef_threshold(settings.TRN_PRED_THRESHOLD)]
loss = weighted_pixelwise_crossentropy_loss(class_weights)

# Compile
model.compile(optimizer=opt, loss=loss, metrics=metrics)

# Load data
The two cells below load the training data. Depending on the value of `USE_GENERATORS` data is either loaded using a generator or loaded into memory. The latter option is useful during development. The former option is almost always required when training with a large(r) number of training images to avoid out of memory errors.

## Option 1: Use generators
Nothing is actually loaded at this point. Only the generators are prepared for use with `fit_generator()`.

In [ ]:
# Prepare generators, only when NOT using cross validation
if USE_GENERATORS:
    print("Using data generators.\n")
    
    # Training set generator
    rdr_train = HDF5Generator_Segment(hdf5_paths[0], hdf5_paths[1],
                                      batch_size=settings.TRN_BATCH_SIZE,
                                      num_classes=settings.NUM_CLASSES,
                                      converter=convert_img_to_pred, 
                                      feat_key=settings.HDF5_KEY)
    gen_train = rdr_train.generator(num_epochs=settings.TRN_NUM_EPOCH)

    print("  Number of training samples: {}".format(rdr_train.num_images()))
    print("             steps_per_epoch: {}".format(rdr_train.num_images()/
                                                    settings.TRN_BATCH_SIZE))


    if USE_VALIDATION_SET:
        # Validation set generator
        rdr_val = HDF5Generator_Segment(hdf5_paths[2], hdf5_paths[3],
                                        batch_size=settings.TRN_BATCH_SIZE,
                                        num_classes=settings.NUM_CLASSES,
                                        converter=convert_img_to_pred, 
                                        feat_key=settings.HDF5_KEY)
        gen_val = rdr_val.generator(num_epochs=settings.TRN_NUM_EPOCH)

        print("Number of validation samples: {}".format(rdr_val.num_images()))
        print("            validation_steps: {}".format(rdr_val.num_images()/
                                                        settings.TRN_BATCH_SIZE))
else:
    print("NOT using generators.")

## Option 2: Load all data into memory
All data is loaded into memory.

In [ ]:
if not USE_GENERATORS:
    print("Loading all data into memory.\n")

    train_imgs = read_images(hdf5_paths[0], settings.HDF5_KEY)
    train_grndtr = read_groundtruths(hdf5_paths[1], settings.HDF5_KEY)
    train_grndtr_ext_conv = convert_img_to_pred(train_grndtr,
                                                settings.NUM_CLASSES,
                                                settings.VERBOSE)
    print("  Number of training samples: {}\n".format(len(train_imgs)))

    # Read the validation set (only when NOT using cross-validation)
    if not USE_KFOLD_CV:
        val_imgs = read_images(hdf5_paths[2], settings.HDF5_KEY)
        val_grndtr = read_groundtruths(hdf5_paths[3], settings.HDF5_KEY)
        val_grndtr_ext_conv = convert_img_to_pred(val_grndtr,
                                                  settings.NUM_CLASSES,
                                                  settings.VERBOSE)
        print("Number of validation samples: {}\n".format(len(val_imgs)))
    else:
        print("Using cross validation, not loading a validation set.")
else:
    print("NOT loading all data into memory.")

# Prepare callbacks
Prepare callbacks used during training:

- TensorBoard: basic TensorBoard visualizations (not always used)
- EarlyStopping: Stop training when a monitored quantity has stopped improving
- CSVLogger: streams epoch results to a csv file
- ModelCheckpoint: save the model after every epoch
- ReduceLROPlateau: reduce the learning when progress halts

In [ ]:
# tb_callb = TensorBoard(log_dir=settings.OUTPUT_PATH + fcn.title,
#                        write_graph=True,
#                        batch_size=settings.TRN_BATCH_SIZE)

cvs_callb = CSVLogger(csv_path, append=False)

if USE_VALIDATION_SET:
    red_callb = ReduceLROnPlateau(monitor="val_loss",
                                  factor=0.2,
                                  patience=5,
                                  verbose=1,
                                  mode="min")
    mc_callb = ModelCheckpoint(model_path,
                               monitor="val_loss",
                               mode="min",
                               save_best_only=True,
                               save_weights_only=True,
                               verbose=1)
    es_callb = EarlyStopping(monitor='val_loss',
                             min_delta=0,
                             patience=settings.TRN_EARLY_PATIENCE,
                             verbose=0,
                             mode="auto")
else:
    red_callb = ReduceLROnPlateau(monitor="loss",
                                  factor=0.2,
                                  patience=5,
                                  verbose=1,
                                  mode="min")

    mc_callb = ModelCheckpoint(model_path,
                               monitor="loss",
                               mode="min", 
                               save_best_only=True,
                               save_weights_only=True,
                               verbose=1)
    es_callb = EarlyStopping(monitor='loss',
                             min_delta=0,
                             patience=settings.TRN_EARLY_PATIENCE,
                             verbose=0,
                             mode="auto")

callbacks = [mc_callb, es_callb, cvs_callb, red_callb]

# Train the model
Execute the training process, either using generators or using data loaded into memory.

## Option 1: Train using generators

In [ ]:
if USE_GENERATORS:
    start_time = time.time()

    if USE_VALIDATION_SET:
        # Prepare callbacks
        print("Training with a validation set, using generators.")
        print_training_info(fcn, model_path, rdr_train.img_shape, rdr_val.img_shape,
                            settings, class_weights, num_patients, opt, loss)

        # Fit the model using generators and a validation set
        hist = model.fit_generator(gen_train,
                                   epochs=settings.TRN_NUM_EPOCH,
                                   steps_per_epoch=rdr_train.num_images()/settings.TRN_BATCH_SIZE,
                                   verbose=2,
                                   validation_data=gen_val,
                                   validation_steps=rdr_val.num_images()/settings.TRN_BATCH_SIZE,
                                   shuffle=True,
                                   callbacks=callbacks)
    else:
        print("Training without a validation set, using generators.")
        print_training_info(fcn, model_path, rdr_train.img_shape,
                        None, settings, class_weights, num_patients, opt, loss)

        # Fit the model using a training set only
        start_time = time.time()
        hist = model.fit_generator(gen_train,
                                   epochs=settings.TRN_NUM_EPOCH,
                                   steps_per_epoch=rdr_train.num_images()/settings.TRN_BATCH_SIZE,
                                   verbose=2,
                                   shuffle=True,
                                   callbacks=callbacks)

    print("\n\nElapsed training time: {:.2f} min.".format(int((time.time() - start_time))/60))

# Option 2: Train using all data loaded in memory

In [ ]:
if not USE_GENERATORS:
    start_time = time.time()

    if USE_VALIDATION_SET:
        # Prepare callbacks
        callbacks = [ModelCheckpoint(model_path,
                                     monitor="val_loss",
                                     mode="min",
                                     save_best_only=True,
                                     verbose=1),
            EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=settings.TRN_EARLY_PATIENCE,
                          verbose=0,
                          mode="auto"),
            CSVLogger(csv_path,
                      append=False)]

        print("Training with a validation set, using all data in memory.")
        print_training_info(fcn, model_path, train_imgs.shape, val_imgs.shape,
                            settings, class_weights, num_patients, opt, loss)

        # Fit the model using a training set only
        hist = model.fit(train_imgs, train_grndtr_ext_conv,
                         epochs=settings.TRN_NUM_EPOCH,
                         batch_size=settings.TRN_BATCH_SIZE,
                         verbose=2,
                         shuffle=True,
                         validation_data=(val_imgs, val_grndtr_ext_conv),
                         callbacks=callbacks)

    else:
        # Prepare callbacks
        callbacks = [ModelCheckpoint(model_path,
                                     monitor="loss",
                                     mode="min",
                                     save_best_only=True,
                                     verbose=1),
                     EarlyStopping(monitor='loss',
                                   min_delta=0,
                                   patience=settings.TRN_EARLY_PATIENCE,
                                   verbose=0,
                                   mode="auto"),
                     CSVLogger(csv_path,
                               append=False)]

        print("Training without a validation set, using all data in memory.")
        print_training_info(fcn, model_path, train_imgs.shape, None,
                            settings, class_weights, num_patients, opt, loss)

        # Fit the model using a training set only
        start_time = time.time()
        hist = model.fit(train_imgs, train_grndtr_ext_conv,
                         epochs=settings.TRN_NUM_EPOCH,
                         batch_size=settings.TRN_BATCH_SIZE,
                         verbose=2,
                         shuffle=True,
                         callbacks=callbacks)

    print("\n\nElapsed training time: {:.2f} min.".format(int((time.time() - start_time))/60))

# Plot/save the training results
Show a plot of the training loss and Dice coefficient by epoch and save it to disk.

In [ ]:
plot_training_history(hist,
                      show=True,
                      save_path=os.path.join(settings.OUTPUT_PATH, prefix),
                      time_stamp=True,
                      metric="dice_coef_threshold")

# Perform pipeline test
Use the trained model on one sample in the training data set. This is just to perform pipeline testing during development.

In [ ]:
# Read training images and ground truths
train_imgs = read_images(hdf5_paths[0], settings.HDF5_KEY)
train_grndtr = read_groundtruths(hdf5_paths[1], settings.HDF5_KEY)

# Predict
predictions = model.predict(train_imgs, batch_size=settings.TRN_BATCH_SIZE, verbose=2)
print("Predictions shape: {}".format(predictions.shape))

# Convert predictions to images
predictions_imgs = convert_pred_to_img(predictions,
                                       threshold=settings.TRN_PRED_THRESHOLD,
                                       verbose=settings.VERBOSE)

# Show a single image, ground truth and segmentation map
show_image(np.squeeze(predictions_imgs[0]), 'Segmentation map')
show_image(np.squeeze(train_grndtr[0]), 'Ground truth')
show_image(np.squeeze(train_imgs[0]), 'Original image')

print("segmentation map {} dtype {}".format(np.max(predictions_imgs[0]),
                                            predictions_imgs[0].dtype))
print("    ground truth {} dtype {}".format(np.max(train_grndtr[0]),
                                            train_grndtr[0].dtype))
print("        original {} dtype {}".format(np.max(train_imgs[0]),
                                            train_imgs[0].dtype))

# Training complete
The trained model is now ready to be applied to test MRI images using `FCN_baseline_test.ipynb`.

# References

*[Long]*: Jonathan Long, Evan Shelhamer, and Trevor Darrell. Fully Convolutional Networks for Semantic Segmentation. In *Corr*, abs/1411.4038, 2014.